## Load ChatGLM-6B

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
# 按需修改，目前只支持 4/8 bit 量化
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).quantize(8).half().cuda()
model.eval()

/home/duhu/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code

## reading a pdf file
code from [pdfchat](https://github.com/bhaskatripathi/pdfGPT/blob/main/api.py)

In [2]:
import re
import fitz

def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text


def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count
    
    if end_page is None:
        end_page = total_pages
        
    text_list = []
    
    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)
        
    doc.close()
    return text_list

## split text to chunks

In [63]:
def text_to_chunks(texts, word_length=400, start_page=1):
    text_toks = [re.sub("[\u4e00-\u9fa5]", lambda x:" "+x.group()[0]+" ", text).split() for text in texts]
    page_nums = []
    chunks = []

    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i : i + word_length]
            if (
                (i + word_length) > len(words)
                and (len(chunk) < word_length)
                and (len(text_toks) != (idx + 1))
            ):
                text_toks[idx + 1] = chunk + text_toks[idx + 1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = '"' + chunk + '"'
            chunk = re.sub("( [\u4e00-\u9fa5] )", lambda x:x.groups()[0][1:-1], chunk)
            chunk = re.sub("([\u4e00-\u9fa5] )", lambda x:x.groups()[0][:-1], chunk)
            chunk = re.sub("( [\u4e00-\u9fa5])", lambda x:x.groups()[0][1:], chunk)
            chunks.append(chunk) #.replace(" ", "")
    return chunks

## 读取文件并分区

In [10]:
pdf_file = "wKg8BmRTWYaAf3B4AAzYfMx0FrI776.pdf"
text_list = pdf_to_text(pdf_file)
chunks = text_to_chunks(text_list)
print(text_list)
print(len(text_list), len(text_list[0]), len(chunks))
print(f"First chunk Example: {chunks[0]}")

['1. 2. 登录 标注平台URL：https://vds.speechocean.com/ 选择企业，输⼊账号和密码后点击登录 领取任务 点击【领取任务】进⼊可领取任务列表，如下图所⽰，可根据项⽬名称和项⽬类型搜索指定任务，点击申请 即可领取任务。 点击【进⾏中任务】，进⼊正在⼯作任务列表，可根据相关条件搜索任务。 ', '3. 开始标注 【进⾏中任务】点击【查看】，进⼊具体的标注界⾯，如下图所⽰。 标注界⾯说明： ● 标注结果区域，区分实体框所使⽤⼯具，⽐如说矩（矩形框）、多边形（多）等，同时通过颜⾊区分 质检状态，⿊⾊（未质检）、蓝⾊（返修且修改）、红⾊（质检驳回）、绿⾊（质检修改）。 ● 数据列表区域，显⽰当前图⽚状态（未标注、已标注、通过、驳回）、已标注实体框数、实体框最新 更新状态。 ● ⼯具集合区域，通过管理端该项⽬涉及业务标注场景来设置⼯具相关参数。 ● 功能区域，查看标注规则，设置⼯具参数，数据统计、快捷键清单、保存、退出、提交。 设置⾥可调整图层透明度，⼯具的样式，对象编辑⽅式等参数。 ', '数据统计⾥显⽰总对象数，标注⽤时和标注速度等信息。 � ⽀持的快捷键 ', '4. �� ● 实体框属性区域，实体框所关联的属性集合，选择不同的属性，实体框填充颜⾊不同，提⾼属性质检 效率。 ● 画布操作区域，⾃适应画布中的图⽚位置和⼤⼩，加减缩放图⽚⼤⼩。 隐藏结果：开启后，显⽰所有标注结果 属性继承：开启后，对象的属性⾃动继承上⼀个对象的属性 开启像素尺：开启后，会显⽰像素尺 ● 图⽚标签区域，可以给图⽚加标签、给图⽚标记⽆效的原因 ● 图⽚翻页区域，上下翻页、跳转相应的图⽚。 ● 批量操作区域，标注记录或画布上按住Ctrl或Shift可多选对象后，可通过锁定按钮或删除按钮批量锁 定或删除处理。 提交任务 跳转到最后⼀张，右上⾓会显⽰提交按钮，点击提交就会提交当前任务。 ', '', '']
6 159 4
First chunk Example: [Pageno.2]"1.2.登录标注平台URL：https://vds.speechocean.com/选择企业，输⼊账号和密码后点击登录领取任务点击【领取任务】进⼊可领取任务列表，如下图所⽰，可根据项⽬名称和项⽬类型搜索指定任务，点击申请即可领取任务。点击【进⾏中任务】，进⼊正在⼯作任务列表，可根据相

- extract the embeddings
    
    ```python
    embeddings = llama.embed_documents(chunks[:10])
    query_embedding = llama.embed_query("这篇文档是讲啥的")
    ```

## load different embedding models

In [5]:
from langchain.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings
llama = LlamaCppEmbeddings(model_path="/data/duhu/llama.cpp/models/7B/ggml-model-q4_0.bin")
hf = HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-chinese")
name2embeddings = {
    "llama": llama,
    "hf": hf
}
embedding_type = "hf"

llama.cpp: loading model from /data/duhu/llama.cpp/models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  =  512.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [6]:
from langchain.vectorstores import FAISS
faiss_index = FAISS.from_texts(chunks, hf) # llama

In [16]:
instructions = [
    "这篇文档是讲啥的？",
    "VDS的标注端怎么登陆？",
    "标注界面数据统计里显示的是什么？",
    "标注界面支持快捷键吗?",
    "怎么提交任务？"
]

In [14]:
topk = 4
history = []
for instruction in instructions:
    docs = faiss_index.similarity_search(instruction, k=topk)
    prompt = "给定材料，请回答问题。\n"
    for index, doc in enumerate(docs):
        content = doc.page_content
#         content = content.replace(" ", "")
        prompt += f"第{index+1}份材料是：{content}\n"
    prompt += f"需要回答的问题是：{instruction}"
    response, current_history = model.chat(tokenizer, prompt, history=[])
    history.append(current_history)
    print(f"="*30)
    print(f"Prompt: {prompt}\n")
    print(f"Q     : {instruction}")
    print(f"A     : {response}")

Prompt: 给定材料，请回答问题。
第1份材料是：[Pageno.2]"环境地址测试http://ai-test.speechocean.com⽣产http://ai.speechocean.com⼀句话识别环境同时⽀持HTTP和HTTPS两种协议。接⼝认证采⽤keyauth的⽅式。apikey可以找算法部⻔志明申请。apikey通过HTTPAuthorizationHeader进⾏身份证认证，认证失败，返回401HTTP状态码。示例：curlhttp://ai-test.speechocean.com/speech/api/v1/health-H'Authorization:申请的apikey'字段类型是否必须备注domainString是语种模型，zh_cnwav_pathFile或URL是⾳频⽂件，⽂件⼆进制流或者URLsegment_lengthFloat否分段⻓度，默认10秒silence_lengthFloat否静⾳⻓度，默认0.5秒add_punctBool否是否添加标点，默认是true字段类型是否必须备注codeInt是错误码，200表示成功，⾮200表示失败messageString是错误描述task_idString"
第2份材料是：[Pageno.3]"是任务IDdata[0].startFloat是开始时间data[0].endFloat是结束时间data[0].textString是识别结果接⼝描述语⾳识别v2请求⽅法：POST请求路径：/speech/api/v2/asr/recognize请求头：⽂件⼆进制流使⽤表单，OSSURL即可以是表单也可以是application/json请求参数：响应参数：流量管理：为便于统计倍速，服务端会根据需求进⾏A/B流量测试。如果强制不进⾏A/B流量测试，可以设置Knative-Serving-TagHTTP头为normal。示例：本地⾳频⽂件识别⾳频URL识别curlhttp://ai-test.speechocean.com/speech/api/v2/asr/recognize\-H'Content-Type:application/json'\-H'Knative-Serving-Tag:normal'\-H'Authorization:apikey'-d\'{"domain":"zh_c

In [29]:
pdf_file = "asr-api.pdf"
text_list = pdf_to_text(pdf_file)
chunks = text_to_chunks(text_list)
print(text_list)
print(len(text_list), len(text_list[0]), len(chunks))
print(f"First chunk Example: {chunks[0]}")

["环境 地址 测试 http://ai-test.speechocean.com ⽣产 http://ai.speechocean.com ⼀句话识别 环境 同时⽀持HTTP和HTTPS两种协议。 接⼝认证 采⽤key auth的⽅式。 apikey 可以找算法部⻔志明申请。 apikey 通过HTTP Authorization Header进⾏身份证认证，认证失败，返回 401 HTTP状 态码。 示例： curl http://ai-test.speechocean.com/speech/api/v1/health -H 'Authorization: 申请的apikey' ", '字段 类型 是否必须 备注 domain String 是 语种模型， zh_cn wav_path File或URL 是 ⾳频⽂件，⽂件⼆进制流或者URL segment_length Float 否 分段⻓度，默认10秒 silence_length Float 否 静⾳⻓度，默认0.5秒 add_punct Bool 否 是否添加标点，默认是 true 字段 类型 是否必须 备注 code Int 是 错误码，200表示成功，⾮200表示失败 message String 是 错误描述 task_id String 是 任务ID data[0].start Float 是 开始时间 data[0].end Float 是 结束时间 data[0].text String 是 识别结果 接⼝描述 语⾳识别v2 请求⽅法： POST 请求路径： /speech/api/v2/asr/recognize 请求头：⽂件⼆进制流使⽤表单，OSS URL即可以是表单也可以是application/json 请求参数： 响应参数： ', '流量管理： 为便于统计倍速，服务端会根据需求进⾏A/B流量测试。如果强制不进⾏A/B流量测试，可以 设置 Knative-Serving-Tag HTTP头为 normal 。 示例： 本地⾳频⽂件识别 ⾳频URL识别 curl http://ai-test.speechocean.com/speech/api/v2/asr/recognize \\ -H \'Content-Type: application/json\' \\ -H \'Knati

In [30]:
from langchain.vectorstores import FAISS
faiss_index = FAISS.from_texts(chunks, hf) # llama

In [31]:
instructions = [
    # "这篇文档是讲啥的？",
    "使用python写一个函数，函数目的是实现本地音频文件识别"
]
topk = 4
history = []
for instruction in instructions:
    docs = faiss_index.similarity_search(instruction, k=topk)
    prompt = "给定材料，请回答问题。\n"
    for index, doc in enumerate(docs):
        content = doc.page_content
        prompt += f"第{index+1}份材料是：{content}\n"
    prompt += f"需要回答的问题是：{instruction}"
    response, current_history = model.chat(tokenizer, prompt, history=[])
    history.append(current_history)
    print(f"="*30)
    print(f"Prompt: {prompt}\n")
    print(f"Q     : {instruction}")
    print(f"A     : {response}")

Prompt: 给定材料，请回答问题。
第1份材料是：[Page no. 2] "环境地址测试http://ai-test.speechocean.com ⽣产http://ai.speechocean.com ⼀句话识别环境同时⽀持HTTP和HTTPS两种协议。接⼝认证采⽤key auth的⽅式。 apikey可以找算法部⻔志明申请。 apikey通过HTTP Authorization Header进⾏身份证认证，认证失败，返回401 HTTP状态码。示例： curl http://ai-test.speechocean.com/speech/api/v1/health -H 'Authorization:申请的apikey'字段类型是否必须备注domain String是语种模型， zh_cn wav_path File或URL是⾳频⽂件，⽂件⼆进制流或者URL segment_length Float否分段⻓度，默认10秒silence_length Float否静⾳⻓度，默认0.5秒add_punct Bool否是否添加标点，默认是true字段类型是否必须备注code Int是错误码，200表示成功，⾮200表示失败message String是错误描述task_id String"
第2份材料是：[Page no. 3] "是任务ID data[0].start Float是开始时间data[0].end Float是结束时间data[0].text String是识别结果接⼝描述语⾳识别v2请求⽅法： POST请求路径： /speech/api/v2/asr/recognize请求头：⽂件⼆进制流使⽤表单，OSS URL即可以是表单也可以是application/json请求参数：响应参数：流量管理：为便于统计倍速，服务端会根据需求进⾏A/B流量测试。如果强制不进⾏A/B流量测试，可以设置Knative-Serving-Tag HTTP头为normal 。示例：本地⾳频⽂件识别⾳频URL识别curl http://ai-test.speechocean.com/speech/api/v2/asr/recognize \ -H 'Content-Type: application/json' \ -H 'Knative-Serving-Tag: norm

## classify the document for asr text
- 方案1: 直接提取每个文件的内容，获得每个文件的领域，最后合并到一起，来确定数据库的类别；
- 方案2: 提取每个文件的内容，获取embedding，聚类之后，再输入到chatglm-6b中，获得类别；

In [33]:
from textgrid import TextGrid
from pathlib import Path
tg_files = Path("asr_data").rglob("*.TextGrid")
tg_files = [str(ele) for ele in tg_files]

In [50]:
def read_one_textgrid(tg_file):
    tg = TextGrid()
    tg.read(tg_file)
    text_tier = None
    for tier in tg.tiers:
        if tier.name == "PEOPLE":
            text_tier = tier
            break
    if text_tier == None:
        return []
    text_all = ""
    for item in text_tier:
        text = item.mark
        if text in ["music", "tts", "noi", "sil", "", "*"]:
            continue
            
        text_all += text + "\n"
    return [text_all]

In [66]:
instruction = "请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开"
import random
random.shuffle(tg_files)
domain_extractor = []
faiss_index = 
for tg_file in tg_files[:100]:
    text_list = read_one_textgrid(tg_file)
    chunks = text_to_chunks(text_list)
    docs = faiss_index.similarity_search(instruction, k=4)
    prompt = "给定材料，请回答问题。\n"
    # history = []
    for index, doc in enumerate(docs):
        content = doc.page_content
        prompt += f"第{index+1}份材料是：{content}\n"
    prompt += f"问题是：{instruction}"
    response, current_history = model.chat(tokenizer, prompt, history=[], temperature=.95)
    # history.append(current_history)
    print(f"="*30)
    print(f"Prompt: {prompt}\n")
    print(f"File:   {tg_file}")
    print(f"Q     : {instruction}")
    print(f"A     : {response}")
    domain = {"prompt": prompt, "file": tg_file, "instruction": instruction, "response": response}
    domain_extractor.append(domain)

Prompt: 给定材料，请回答问题。
第1份材料是："喂您好这里是安居客新房频道的您之前是有关注过正荣财富中心这个楼盘吗哦对呀怎么啦我们是在本周的话有一个大型的线下的看房活动此次是在就是五月十号早上八点半在五一广场正门口就直接搭去那边那本次线路的话是一共有四个楼盘可以参观有融汇温泉城这个是在静安区飞狐孙年温泉小镇度假区内的然后这户型的话是有三十五到六十二平方的SOHO清苍公寓还有是在凤翔湖滨世纪的话就是在敏阳县的京西东大道六十六号参考价格是八千五一平然后现仅剩户型的话也是SOHO三十六到九十二平方还有是正荣财富中心是府昌大道与乌龙江大道交汇处西北侧按场优惠是十二个优惠一个点户型的话是七十八八十九一百一十平方也都有还有最后一个是中铁城也是在闽行县的金居正干红路主推户型是七十八平到一百十八平的女士您看一下您那天有时间方便过去看吗我不知道到时候再看吧因为我们这次活动的话也是有免费大巴接送客户去的并且这次活动的优惠力度也是挺大的是有优惠一个点的如果个人去的话是没"
第2份材料是："有的所以如果您这边有时间的话就可以就是抽空去了解一下也是可以的嗯嗯好吧我再看吧啊现在没没那个需要哈哦好的那我们这边安居客先祝您生活愉快再见嗯"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/50h/HT1103_50h质检完成版/100133028280140506.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 领域：房产、安居客、新房、线下看房活动、优惠。
Prompt: 给定材料，请回答问题。
第1份材料是："是吧对那这样家长咱们在这个十九号上午啊就有咱们那个高考语文阅卷组的张建老师呀针对咱们考场作文有一个系统的一个精讲讲呢就是咱们那个高考作文就怎么写怎么能够就是得高分然后呢还有政策的一些方面这个是咱们家长来听一下就是给孩子做一个把关吗嗯您看您那个十九号上午方便吗嗯十九号对十九号上午然后这几天还还有别的那什么您这边就是给孩子报了别的班了是吗嗯没有没有但是嗯嗯现在啊有别的那个那什么吧诶有别嗯是这样的家长咱们那个张健老师呀也是多年来参加咱们高考语文这个阅卷工作对这个考场作文这个命题呀方向呀字里呀有独到的见解他就是带着学生嗯都是考进那些重点大学像这清华北大呀人大都很多就是着重是提高

Prompt: 给定材料，请回答问题。
第1份材料是："喂哎鲁怡妈妈喂您好您好哎咱是鲁怡妈妈吗诶诶您说您说呃是这样我是不是我是张玉张老师迎接的那个然后嗯是嗯现在就是方便说话吧嗯说吧正好我在找你呢嗯是这样然后就是孩子月考成绩出来了吗在一块儿觉得他有没有回家说然后老师这一块讲的怎么样月考成绩该出来了吧他这一块学的他有没有回家说我们老师这一块讲的怎么样嗯月考成绩该出来了吧我还没听他说呢没听他叨叨呢哦没说呢哈那就是回家有没有说这块学的怎么样现在老师讲的这一块他都能吸收吗难易程度然后都可以吗还可以还可以是吧我看他还行对还还可以接受哦但是他没有具体跟我说这事今天晚上回去问问他吧啊成咱回去有什么问题然后慢就是看看也就是说看给孩子聊一下那那个明天明天孩子上课吗明天不上明天的课在下周上嗯对就是说呢就原来就在初二的时候在他那个那个那个那个那个那个数学老师姓什么来着刚才还想起这会而忘了蒋老师您再说一遍蒋老师蒋文祥蒋那初二那个蒋老师那是那个还带他们吗数学嗯数学他现在不"
第2份材料是："点嗯哦那那那那以后再说吧以后要是我们要是报一对一的再说好吧成嗯嗯月考成绩然后还是这个学校的估计还没出来呢有的都出来了我就看看这次孩子考的怎么样然后在这一块学的什么样没什么事没其他事嗯他没跟我说这事呢完了再说吧好成好的好的嗯哎好的再见嗯再见"
第3份材料是："带这个然后他只带一个初一的现在我这会听不见我我找一地方啊没事没事那要不我晚会给您晚上打吧您下班了以后再给再打也成没没事我现在已经下班了我就是得找一个声音小点的我听不见行那那那老师怎么了他他现在一般的话因为年龄大了嘛然后他身体也不好他一般就是带个一对一然后带的一对一的多他班课的话有的上课的话有几个孩子他就是一直得站着他身体有时候受不了所以然后噢一对一是吧对对嗯行以后再跟这孩子说吧嗯成他还想让那个老师给他讲课是吧对他还是就愿意接受那个老师毕竟说是习惯了吧嗯你说其实他这老师也还行我就是问问帯手问一下问一下那个老师怎么样哦也成嗯没别的意思嗯如果要是这这一块现在上课的这几个老师给咱孩子上课的有什么问题的话然后看看哎我给老师这样沟通一下因为要是蒋老师呢现在是他确实然后也干了这么长时间孩子一致反映他挺好的然后回头的时候看看嗯带班课的话他确实挺吃力的现在然后身体这一块有的时候受不了所以他就是带一对一会多一"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔

Prompt: 给定材料，请回答问题。
第1份材料是："知道了然后慢慢他就多背点单词然后对于语法的东西呢还有就是说词词就是词组啊这些的嗯然后这个英语成绩就很不错了那他那个像阅阅读啊还有作文啊这块呢阅读语文他就读吧我反正我催着让他多看会多多那个看点书嗯多把语文知识呢诗词那东西多看看嗯嗯还老催着他吧反正你不催他孩子始终是孩子吗他也没自觉性哦学习自觉性还是稍微差点对嗯嗯行您可以到时候看看孩子这个月考成绩然后那个看哪科学的比较那个就是他英语这方面的薄弱点薄弱点啊行VB那打扰您了哎谢谢啊好哎再见哎再见"
第2份材料是："喂哎您好是德枫妈妈是吧对哎打扰您了我是那个金翰教育的老师给您打电话想问一下孩子最近学习怎么样啊还行吧还行吧他们学校月考了吗月考了分数没下来呢还哦那孩子回来给您说考的怎么样啊感觉嗯感觉还行他说啊感觉还行就是做题啊什么的挺都挺顺利的嗯嗯嗯那孩子那个就是还在那个就是那做补习吗嗯对嗯效果怎么样啊一直在那补着呢他就现在补英语不补了嗯那那个数理化不补那个一对二了嗯不补了哦那是假期假期补的对对对现在不补了嗯嗯那那个英语现在还是一对一的给他补是吧对给他补着呢嗯是补到什么时候啊再看吧嗯再看我也不知道补到什么时候他英语提上来了就不用补了嗯嗯那现在孩子这个对英语这个兴趣怎么样啊反正到高中我觉得感觉好点了就不像初中那么费劲了就嗯就是这最起码的这英语也块反感了也不不那个他最起码他自己知道写知道知道那个完成作业了嗯嗯嗯好那还挺不错的那孩子就是说从从兴趣上然后对这个学习啊自己知道就是哪些就是语法语嗯这个就是考点什么的都"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/100h/HT1114--100h/2628fcbf-5b88-45c0-b934-8d58b1be291c.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 1. 英语
2. 语文
3. 阅读
4. 作文
5. 数学
6. 补习
Prompt: 给定材料，请回答问题。
第1份材料是："爸就给我打电话了说怎么有考试呀问我知不知道这事我说我给你问一下然后昨天晚上都十点了给我打电话说孩子我给孩子发信息也没回说孩子是不没考好啊怎么怎么地的他说要不然你就帮我问问老师因为孩子本身心思重问问老师怎么回事考的不好能不能学了你说你

Prompt: 给定材料，请回答问题。
第1份材料是："喂喂那郭老师方便接听电话吗啊是您是哪是那个京翰人大小区的薛乔啊梁老师然后我想问老师您那个明天有点吗嗯你是哪人谁京翰京翰人大校区啊人大校区的嗯嗯然后老师您那个明天有点吗呃您呃怎么的明天是不有课对然后我们这明天有个孩子过来想上想来上课然后您看您这边明天有点吗啊你那是什么孩子这个是高三标准的标准嗒对标准的我现在先不想接了嗯老师您就是假如说能帮忙带带一下呗现在性格学生特别不容易您就帮忙带一下呗希望这个孩子能报中级的但是确实是因为我现在这个因为我这两天就说这个现在时间段吧也比较忙因为现在我也带着一个那个孩子呢我知道您是那个十到十二带着一个那个李玉晗是吧对对对然后您就帮忙带一下呗再加一个然后咱这边确实是真有学生就给您排着这种然后不管是不就只要是有学生就给您打电话但是他这个时间我要是那什么就太实在是太那什么了因为这个时间要想都能排在上午吧我还凑合要不然下午的我有别的事实在是安排不开了就呃这就临时一次一到"
第2份材料是："三行吗啊就临时一次就这次先把这次上了然后下周的话呃咱到时候再临时调一下点咱把点时间是什么时候这孩子这世间就因为呃那个语文的话她八到十是语文是那个老师给重新临时调的点啊到下一周的话应该能调开是吗对然后您就明天就帮忙带一下一到三呗一点到三点对对因为第一次您是您是哪位老师我是梁老师协调梁老师啊梁老师庄老师说那孩子呀嗯不是这是另一个孩子这是另一个孩另一个学生十九中的啊男生女生呀这是呃这个是男孩啊是吗嗯对对您放心绝对不就我就临时带一次如果以后有时间能腾出上午我就带商量一下下午肯定我就我就不带了行好嘞行那成那您麻烦老师您明天帮我带这个孩子行吗啊行先找你呗谢谢啊嗯郭老师拜拜诶拜拜"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/100h/HT1114--100h/64527596-1d6d-434f-bd99-a0ede3c65b26.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 领域：教育/学生辅导
Prompt: 给定材料，请回答问题。
第1份材料是："专业性的了是在我的这个专业以外的知识了对不对我这边不是这个银行这边的呀对不对利息怎么算的我也不知道呀啊哎好我知道那这次说礼拜六对吧但是这个时间肯定没有

Prompt: 给定材料，请回答问题。
第1份材料是："您好先生这边是安居客新房频道致电过来的了解先生有关注青岛这边的楼盘是吗嗯对嗯我们安居客呢会不定期的给就是有购房需求的客户推荐一些新房源组织那个看房团购活动嘛此次推荐的呢是中海临安府就是长途汽车站这边的一个楼盘那是礼拜六九点半在新业广场肯德基门口集合邀请先生呢一起过来了解一下您看时间上方便吗嗯那边均价多少钱一万两千六在杭州路二十八号对什么杭州路杭州路是吧在杭州路二十八号楼盘地址然后买房的话呢就是说优惠后会在一万一到一万二之间吧现在的优惠政策是一百平米以下经典小户型交一万五最低可享受九四折优惠嗯就是说您就是决定看好了这个房源嘛那这个房子呢是在一百平以下的是然后您先交一万五然后呢就是说买房的时候呢总价上打九四折首付多少钱呢首付啊因为它有高层小高层然后也有那个十十三幢的墅区别墅这个的话嗯那如果您您是看的那个最低首付呢是三十五万四别墅是两百四十万到三百二十万之间嗯还有别的房源吗便宜点的便宜点的嗯嗯就"
第2份材料是："是这个首付也嗯您是说这个楼盘还是说其它地段的楼盘其它地段的楼盘其它地段的呀这次的话只组织了一个专场看房就是针对这个楼盘的看房嗯您关注的是哪个区呢到时候我我再跟您就是说留意一下有活动我再致电给您城阳城阳那一块吧同样的一块也也是长途汽车站周边的不用正阳路上就行了嗯什么路正阳路重阳正阳路正重阳路对正阳路噢行也就是城阳区这块的吗城阳区跟你说了城阳区了噢城阳区是吧因为我这个楼盘是四方区我一直噢是正阳路上的是吧买个多大的户型八十到九十吧两房的或小三户嗯两房的或小三房的对嗯好了解了那单价呢您说这个楼盘嗯就说有点贵然后您觉得单价在多少能接受我这边好帮您筛选那起码是越便宜越好吧越便宜越好那就说有没有一个就是标准呢就说八九千一万左右嗯七八千吧七八千是吧嗯嗯好好了解了行那我这边再帮您留意好吧我们下次在城阳这边有看房活动有合适您的房源再给您致电行嗯那先生怎么称呼呢免贵姓李木子李对行那李先生不打扰您了安居客祝您生活"
第3份材料是："愉快好再见噢好"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/50h/HT1103_50h质检完成版/200120999590140527.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     :

...

Prompt: 给定材料，请回答问题。
第1份材料是："子各科的情况因为毕竟咱们明年不就参加中考了嘛然后现在千万别让孩子有这个缺九的科目哦这些我都知道呃那那您这样就是假期的时候呃找假期的时候他们应该放我看啊他们也会补课大概是补一个星期吧然后剩下有二十多天的时间到那个时候可以给孩子集中集中的查漏补缺一下也可以嗯那现在就是说孩子他平时呀或者周末呃都是自己学吗有没有老师给他辅导呢没有呃就孩子自已学是吧对昂那您这块还是要多关注关注的因为毕竟来说他们现在呀这个学习这课程啊进度比较快学校老师在讲在赶课在赶进度嘛所以说这个时候呢呃那么孩子呢就是说快是正常的但是孩子消化不了呢可能也比较正常那么也就是说孩子的学习习惯呀包括学习方法这块就很重要了我明白昂然后咱也看一看吧然后假期的时候就是孩子放假时间长一些然后看看有没有时间要有时间的话咱们也可以让孩子昂这边学一学好吧好嘞诶好嘞啊那您先忙吧稍后再联系啊郑梦妈妈嗯好再见诶"
第2份材料是："喂诶你好廖梦妈妈诶我是京翰望京校区梁老师就是跟您通过电话嗯也是想问问就是说小孩呃这边学校有没有开家长会呀哦干什么呃上次就是听您说选就是孩子他们学校现在不是补课嘛然后呢也是刚约考完然后您说小孩的成绩也可以我说正常就是说开呃就考完之后家长就是说学校不是开家长会嘛我想那个问一下就是呃咱们这边学校开了没有啊没有没有嗯那什么时间开呢说了吗没有没说哈不知道我不知道呢还还不知道呢哈呃然后那个呃我说想看一看就是小孩子现在各科成绩怎么样啊成绩怎么样你们那学校太远我去不了太远了您住哪块呀我住哪他都那什么他现在就是单休日就一那然后再去学校再上课就半天了作业都写不完了弄不了去不了孩子是住校吗还是走读呀不不住校啊住校干嘛呀不住校哈呃你像那个就九一的那个李杰我不知道是不是他们班的嗯好像不是吧我没听他说过这小孩名字是吧那李杰他也是九一的他家住中坝然后现在也在这边学呢其实现在距离呀远近不是最主要的问题现在咱们还是说看看孩"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/100h/HT1114--100h/05e586db-0141-4167-9837-1df625798321.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 对话涉及到的领域有：中考、家长、学校、科目、成绩

Prompt: 给定材料，请回答问题。
第1份材料是："以给您安排一个一线的在职老师帮助孩子进行点拨或者是进行例题精讲或者是知识模块的串讲嗯咱们这体验课呢就是十八元然后咱们家长可以在网上操作是在大众点评网上您可以自己去团购然后咱们上课呢是来校区来上课所以就是嗯想问问咱们孩子这边就是这次月考有没有哪一科目就是觉得还是有一点问题呢吗嗯他现在没有哦那现在暂时没有是吗对哦那孩子就是那就是他就是那这次考试嗯他这次考试是没有那就之前所学的知识模块有需要咱们这边老师进行点拨的吗没有没有是吧那没关系就是咱们只是把这活动通知到您如果咱们孩子后期这边就是如有什么需求的话那我们可以再联系那我顺便问一下那咱们孩子近期会有家长会家长会吗不知道因为一个礼拜才回来一回呢不知道哦就是咱们这边哦咱们这边还没有接到通知是吧对哦那行那可以嗯那这样就是如果以后有什么关于学习方面的事情那我们随时保持联系好吧那今天打扰到你啦嗯好好好好再见哎哎好好再见哎再见嗯"
第2份材料是："您好哎您好是剑傲的妈妈是吧对哎您好这边是金翰教育牛老师家长您好是这样就是孩子现在应该是上高二了是吧对呃那咱们孩子就是在首师大附中就最近不是又月考吗那咱们孩子这边考得怎么样呀嗯考得还行呃他是到高二以后学文学理了呢这次考大概是什么名次呢嗯学的文哦学文了是吗哦那咱们孩子现在是排什么名次这次月考这次十七年级哦十七是吧嗯那孩子跟您沟通过了吗就是这次嗯月考孩子觉得难易程度怎么样呀嗯他没说呢还没说呢是吧那您这次还没有看到孩子呢是吧只是老师发信息是吗嗯对嗯嗯然后是这样就是剑傲妈妈咱们今天给您打电话呢是因为咱们这边是金翰教育的北京有十四年但是在门头沟这边很多家长并不了解所以咱们这边呢也是就是集体给家长打个电话做一下回访也是做下品牌推广咱们就是有一个针对这次月考的有一个一对一的然后六十分钟十八元的一个试听课就是孩子比如说他嗯这次考试有哪一科目觉得是薄弱科目或者是某一种题型还是暂时还没有解题思路的话那咱们这边可"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/100h/HT1114--100h/2341f2c8-125c-4916-98eb-974bc2a9ca80.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 1. 教育领域
2. 家长/老

Prompt: 给定材料，请回答问题。
第1份材料是："哎你好哎您好这里是安居客新房频道给您致电的您好嗯你好啊请问您是不通过了楼盘四零零电话关注过宗盛新河湾的楼盘啊嗯对哎女士请问您是比较关注固安这边的楼盘对吗对啊是这样的就本次给您致电是想邀请您参加就是在我们本周六也就是五月十号上午九点固安这边有五个楼盘组织了大型的线下看房活动分别是庆元山十号我没时间啊女士是这样的考虑到您的时间比较忙所以本次活动特地安排在周六的而且我们这五个楼盘越是周六日我们越没时间奥如果您实在没有时间的话可不可以让你家人或者你的朋友代替您参加一下到时候在您买房的时候可以给您更多的意见和见解不行我们家孩子周六要上课然后他爸爸要去送他我要有一个那个婚礼要去参加我十号那一天奥那呃这边冒昧的问一下女士您就打算在固安这边买房了对吗对呃其他区域您都不考虑了吗不考虑那您打算买多大的呢七十到八十平的就是两居室的对吗对看你理想的价位大概在多少呢越便宜越好喽嗯女士是这样的因为我们安居客这边是不定"
第2份材料是："时为导航用户推荐新房信息的嘛根据您刚刚所说的这些呃这些我们这边有适合您的房源我们会及时通知您的好吧嗯好谢谢啊哎好的那您这边先忙我就不打扰了安居客祝您生活愉快再见哎好再见"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/50h/HT1103_50h质检完成版/100158058180140507.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开
A     : 1. 安居客新房频道
2. 楼盘
3. 线下看房活动
4. 购房者
5. 新房信息
Prompt: 给定材料，请回答问题。
第1份材料是："您好我这边是安居客新房频道的您之前是否关注过郑州这边的楼盘呢郑州这边的楼盘有关注过吗没有啊是这样子的我们在本周日的上午八点半呢在紫金山立交桥下人民会堂西门口这边是有个免费的大巴接送您过去聊这个叫观澜晋的这个楼盘您看就是说到时候可不可以和我们一起就出席这个活动呢观澜晋他是哪一片的呃这边的话是属于那个龙湖这里的龙湖那个祥云路与红浦路交口向西两百米北这样子的位置嗯他是属于洋房还是高层啊呃小高层和高层呃那先不考虑吧呃但你可以先过来了解一下嘛呃因为毕竟的话就是说跟我们一起去的话是有个优惠活动的嘛是三千抵一万五然后在这个优惠就是在

Prompt: 给定材料，请回答问题。
第1份材料是："价优惠五点五万呢户型的话是七十二平到一百三十二平不等的两房到四房嗯嗯嗯嗯有些我看过了有些我知道嗯我这个礼拜可能去不了这个礼拜没有时间嗯嗯对对嗯嗯但我觉得优惠力度嗯您说你你是不是就就这个礼拜四活动是吧嗯这周的话汉阳县这边就这个礼拜但是我不确定下个下个礼拜有没有汉阳县的嗯嗯汉阳县的那边知道了我有时间过去看看我前两天就看过了看过好多那您该怎么称呼呢贵姓免贵姓龚龚女士龙共龚那您这边的话主要是我看您之前也看到很多嘛然后然后您现在话主要考虑汉阳还有哪个区域的就是这边的吧湛普那边的乔普嗯湛潽湛潽只有就就湛潽开发区这边的嗯对对对对然后考虑两房还是三房呢两房三房都可以主要是一个是看地段一个看价位看一个看小那个开发商开发商也嗯开发商那是啊正规的各方面人都不错的嗯哎我嗯龚女士我们这边的话有个很大开发的盘这次看的有恒大御景湾在国博五百米的旁边呢什么御景湾呢恒大御景湾嗯这个没有这个没有嗯怎么了这这个楼盘怎么了不是这"
第2份材料是："你好我这边是安居客新房频道的您之前有关注过万科金域蓝湾这个小区是吗你是哪里呀呃你好我这边是安居客新房频道的是这样的我们这边周六呢有个看房团是看汉阳这边的五个小区一起看的你这周六有空过来看一下吗哦看哪个地方的恩像金色港湾在市外国语学校对面的均价七千五一平然后的话它现在主要在做的户型是七十五到八十五平的享受两万抵四万的优惠还有一个是少量的一百八十七平的府井大平层嗯连投金色港湾是吧嗯这个是金色港湾的就金色港湾嗯连投金色港湾的话在博学路六号嗯我知道那我看过了然后的话它高层它价格它没给我们这边没给我们确定说然后洋房的话是八千七百元每平的优惠呢是两千抵两万嗯然后你还有哪的嗯还有一个就是金色港湾双湖泊岸是在三环线与芳草路交汇处的嗯七千七百元每平这是高层的嗯优惠的话在七十九到一百二十三平米认购的话是享受五千五千享受两万优惠的嗯嗯嗯嗯嗯嗯广电兰亭时代它这个是在江城大道与四新南路交汇处的均价是七千六百元一平总"
第3份材料是："个没去看呢嗯这个是八千九百八十八元每平起的精装修的一千六百元每平精装修标准嗯嗯嗯在哪个位置这个是在鹦鹉大道与汉新大道交汇处的嗯嗯嗯嗯嗯我要是去了给你们联系的电话就你电话你这不是上海的吗嗯我这个不是我这个是一个固话的网络外播的所以说没法联系的哦哦哦行行行不能去周六去不了是这样的嗯嗯行那我们这边帮您争取不了

Prompt: 给定材料，请回答问题。
第1份材料是："啊女士哎你好不好意思打扰您了我这边是安居客新房频道给您致电的请问一下女士您目前这边有在关注就是花都这边的楼盘是吧是您吧嗯嗯嗯呃刚好这个星期六上午的九点钟呢我们这边推出了一个清远线路的一个楼盘这边刚好有花都市里的然后有共有四个给到您参考你现在方便听一下吗这四个楼盘嗯星期六啊多少号啊啊六月二十八号上午的九点钟在体育西路H出口这边有大巴车接送哦不用了那天那天我都没空哦那你可能没时间是吗时间啊女士这样的就是你现在方便听一下吗就有万科华府这楼盘是六千八一个平方有个优惠给到您是一元抵四点五万元然后第二个是北部万科城它是五千五一个平方优惠是一点五万元抵五万还那吗什么那边是清远的吗呃这边是广州的花都市里的在广清高速这边那个楼盘北部万科城哦对因为北部万科城这个楼盘一直关注度还是蛮高的因为它附近这边就是嗯就交通还是比较见便捷的而且这边价格也是比较就是性价比蛮高的也值得您参考一下这边第三个很大一物产之间是六千五"
第2份材料是："一个平方优惠是九九折然后中国美林湖你边应该听说过他是在花都那个市里这边的是六千八一个平方优惠是二万抵九五折刚好本周六有这样的一个免费大型的看房活动嘛也不想让您错过这样一个看房机会你可以跟随我们大巴车到现场去了解一下您看这边可否方便呢嗯嗯嗯嗯嗯那就往后再看吧嗯啊你可以就是是还不确定是吧嗯嗯嗯哎好的没关系女士那是这样的就是您目前除了考虑在清远这边购房有考虑过其它地段吗间有看啊呃就清远这边是吧有考虑过在番禺增城萝港从化花都这边购房呢都有看哪都有看是吧呃那女士是这样的就是你有考虑过在番禺增城吗啊买啊清远萝港这边购购房吗因为我们这个星期有这样的一个线路的都有这个线路的嗯我清远再看看吧哦可能再看看是吧边就先不打扰冒昧问一下怎么称呼您您贵姓呢女士您贵姓嗯嗯我姓林啊林女士是吧呃好林女士那您先先忙我这边先不打扰了您可以先嗯稍后再去了解一下如果觉得还不错就去参加我们这个活动好吗好的啊那好这边先不打扰您了安居客"
第3份材料是："祝您生活愉快嗯再见嗯好嗯拜拜"
问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开

File:   asr_data/50h/HT1103_50h质检完成版/1001027916B0140625.TextGrid
Q     : 请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开


In [69]:
import json
print(json.dumps(domain_extractor, indent=2, ensure_ascii=False))

[
  {
    "prompt": "给定材料，请回答问题。\n第1份材料是：\"喂您好这里是安居客新房频道的您之前是有关注过正荣财富中心这个楼盘吗哦对呀怎么啦我们是在本周的话有一个大型的线下的看房活动此次是在就是五月十号早上八点半在五一广场正门口就直接搭去那边那本次线路的话是一共有四个楼盘可以参观有融汇温泉城这个是在静安区飞狐孙年温泉小镇度假区内的然后这户型的话是有三十五到六十二平方的SOHO清苍公寓还有是在凤翔湖滨世纪的话就是在敏阳县的京西东大道六十六号参考价格是八千五一平然后现仅剩户型的话也是SOHO三十六到九十二平方还有是正荣财富中心是府昌大道与乌龙江大道交汇处西北侧按场优惠是十二个优惠一个点户型的话是七十八八十九一百一十平方也都有还有最后一个是中铁城也是在闽行县的金居正干红路主推户型是七十八平到一百十八平的女士您看一下您那天有时间方便过去看吗我不知道到时候再看吧因为我们这次活动的话也是有免费大巴接送客户去的并且这次活动的优惠力度也是挺大的是有优惠一个点的如果个人去的话是没\"\n第2份材料是：\"有的所以如果您这边有时间的话就可以就是抽空去了解一下也是可以的嗯嗯好吧我再看吧啊现在没没那个需要哈哦好的那我们这边安居客先祝您生活愉快再见嗯\"\n问题是：请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开",
    "file": "asr_data/50h/HT1103_50h质检完成版/100133028280140506.TextGrid",
    "instruction": "请用最多五个词总结一下对话涉及到的领域，输出结果用、隔开",
    "response": "领域：房产、安居客、新房、线下看房活动、优惠。"
  },
  {
    "prompt": "给定材料，请回答问题。\n第1份材料是：\"是吧对那这样家长咱们在这个十九号上午啊就有咱们那个高考语文阅卷组的张建老师呀针对咱们考场作文有一个系统的一个精讲讲呢就是咱们那个高考作文就怎么写怎么能够就是得高分然后呢还有政策的一些方面这个是咱们家长来听一下就是给孩子做一个把关吗嗯您看您那个十九号上午方便吗嗯十九号对十九号上午然后这几天还还有别的那什么您这边就是给孩子报了别的班了是吗嗯没有没有但是嗯嗯现在啊有别的那个那什么吧诶有别嗯是这样的家长咱们那个张健老师呀也是多年来参加咱们高